In [1]:
! pip install pytorch-ignite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
import os
import random
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from ignite.metrics import Rouge
from pandas.core.common import random_state
from PIL import Image , ImageFilter
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import GPT2Tokenizer, VisionEncoderDecoderModel, ViTImageProcessor , BertTokenizer
from typing import Any, Dict, List

In [3]:
seed = 48763

np.random.seed(seed=seed, )

random_state(state=seed, )

random.seed(a=seed, )

torch.manual_seed(seed=seed, )
torch.cuda.manual_seed(seed=seed, )
torch.cuda.manual_seed_all(seed=seed, )
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [34]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained("nickmuchi/vit-finetuned-chest-xray-pneumonia", "bert-base-cased").to(device)
######################################################## bert based model
tokenizer = BertTokenizer.from_pretrained(############## bert based model
    "bert-base-cased" , cache_dir= '/cache'
)

print(tokenizer.all_special_tokens)

tokenizer.add_special_tokens({
    "bos_token": "[beginoftext]",
    "pad_token": "[PAD]",
    "eos_token": "[EOS]",
})

tokenizer.bos_token_id = 400404

model.config.decoder_start_token_id = tokenizer.bos_token_id
model.generation_config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

model.decoder.resize_token_embeddings(len(tokenizer))
processor = ViTImageProcessor.from_pretrained("nickmuchi/vit-finetuned-chest-xray-pneumonia" , cache_dir='/cache')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at nickmuchi/vit-finetuned-chest-xray-pneumonia and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.se

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


In [5]:
class CXIRGDataset(Dataset):
    def __init__(self, data: List[Dict[str, Any]]) -> None:
        super(CXIRGDataset, self).__init__()
        self.data = data

    def __getitem__(self, index: int) -> Dict[str, Any]:
        return self.data[index]

    def __len__(self) -> int:
        return len(self.data)

In [6]:
max_length = 256

def train_dl_collate_fn(one_batch_data: List[Dict[str, Any]]):
    names = [one_data["name"] for one_data in one_batch_data]

    pixel_values = processor(
        images=[one_data["image"] for one_data in one_batch_data],
        return_tensors="pt"
    ).pixel_values

    label_ids = tokenizer.batch_encode_plus(
        batch_text_or_text_pairs=[
            (tokenizer.bos_token + one_data["text"] + tokenizer.eos_token) for one_data in one_batch_data
        ],
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    ).input_ids

    return names, pixel_values.to(device), label_ids.to(device)

def valid_dl_collate_fn(one_batch_data: List[Dict[str, Any]]):
    names = [one_data["name"] for one_data in one_batch_data]

    pixel_values = processor(
        images=[one_data["image"] for one_data in one_batch_data],
        return_tensors="pt"
    ).pixel_values

    label_ids = tokenizer.batch_encode_plus(
        batch_text_or_text_pairs=[
            (tokenizer.bos_token + one_data["text"] + tokenizer.eos_token) for one_data in one_batch_data
        ],
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    ).input_ids

    return names, pixel_values.to(device), label_ids.to(device)

In [9]:
train_data = []

report_path = "/content/train/reports.xlsx"
report_df = pd.read_excel(report_path)

image_dir_path = "/content/train/image"
for image_name in os.listdir(image_dir_path):
    image = Image.open(os.path.join(image_dir_path, image_name))
    image= image.filter(ImageFilter.DETAIL)############################################################ additional operation: image filter
    image_array = np.array(image).astype(np.float32)################################################### additional operation: normalize
    normalized_array = (image_array - np.min(image_array)) / (np.max(image_array) - np.min(image_array))
    image = Image.fromarray((normalized_array * 255).astype(np.uint8))
    if image.mode != "RGB":
        image = image.convert("RGB")

    text = report_df[report_df["name"] == image_name[:13]]["text"].values[0].replace("_x000D_", "\r")

    train_data.append({
        "name": image_name[:13],
        "image": image,
        "text": text
    })

train_dataset = CXIRGDataset(train_data)

In [12]:
valid_data = []

report_path = "/content/valid/reports.xlsx"
report_df = pd.read_excel(report_path)

image_dir_path = "/content/valid/image"
for image_name in os.listdir(image_dir_path):
    image = Image.open(os.path.join(image_dir_path, image_name))
    image= image.filter(ImageFilter.DETAIL)############################################################ additional operation: image filter
    image_array = np.array(image).astype(np.float32)################################################### additional operation: normalize
    normalized_array = (image_array - np.min(image_array)) / (np.max(image_array) - np.min(image_array))
    image = Image.fromarray((normalized_array * 255).astype(np.uint8))
    if image.mode != "RGB":
        image = image.convert("RGB")

    text = report_df[report_df["name"] == image_name[:13]]["text"].values[0].replace("_x000D_", "\r")

    valid_data.append({
        "name": image_name[:13],
        "image": image,
        "text": text
    })

valid_dataset = CXIRGDataset(valid_data)

In [38]:
lr = 1e-5
epochs = 30
optimizer = AdamW(params=model.parameters(), lr=lr)

train_batch_size = 1
valid_batch_size = 1
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    collate_fn=train_dl_collate_fn
)
valid_dataloader = DataLoader(
    dataset=valid_dataset,
    batch_size=valid_batch_size,
    shuffle=False,
    collate_fn=valid_dl_collate_fn
)

rouge = Rouge(variants=["L", 2], multiref="best")

In [36]:
def evaluate(model: VisionEncoderDecoderModel, epoch: int) -> Dict[str, float]:
    model.eval()

    pbar = tqdm(valid_dataloader)
    pbar.set_description(f"Evaluting Epoch: {epoch + 1}")

    with torch.no_grad():
        for names, pixel_values, label_ids in pbar:
            predictions = model.generate(pixel_values,max_length=100)

            _predictions = tokenizer.batch_decode(
                predictions,
                skip_special_tokens=True
            )

            _labels = tokenizer.batch_decode(
                label_ids,
                skip_special_tokens=True
            )

            print(f"Names      : {names}")
            print(f"Predictions: {_predictions}")
            print(f"Labels     : {_labels}")
            print()

            for _prediction, _label in zip(_predictions, _labels):
                split_prediction = _prediction.split()
                split_label = _label.split()

                rouge.update(([split_prediction], [[split_label]]))

    return rouge.compute()

In [35]:
def generate_result(model: VisionEncoderDecoderModel, epoch: int) -> Dict[str, float]:
    results = []
    model.eval()

    pbar = tqdm(valid_dataloader)
    pbar.set_description(f"Evaluting Epoch: {epoch + 1}")

    with torch.no_grad():
        for names, pixel_values, label_ids in pbar:
            predictions = model.generate(pixel_values,max_length=100)

            _predictions = tokenizer.batch_decode(
                predictions,
                skip_special_tokens=True
            )

            _labels = tokenizer.batch_decode(
                label_ids,
                skip_special_tokens=True
            )

            print(f"Names      : {names}")
            print(f"Predictions: {_predictions}")
            print(f"Labels     : {_labels}")
            results.append(_predictions)
    return _predictions

In [39]:
min_loss = float("inf")
rouge_result = []
for epoch in range(epochs):
    model.train()

    pbar = tqdm(train_dataloader)
    pbar.set_description(f"Training Epoch [{epoch + 1} / {epochs}]")

    lost_list = []
    average_loss = 0

    for _, pixel_values, label_ids in pbar:
        optimizer.zero_grad()

        loss = model(
            pixel_values=pixel_values,
            labels=label_ids
        ).loss

        lost_list.append(loss.item())
        average_loss = np.mean(np.array(lost_list))
        loss.backward()
        optimizer.step()

        pbar.set_postfix(loss=average_loss)

    if average_loss < min_loss:
        min_loss = average_loss
        torch.save(model.state_dict(), f"/content/outputs/best_checkpoint.pth")
    result = evaluate(model=model, epoch=epoch)
    rouge_result.append(result)
    print(f"Rouge-2 score on epoch {epoch}:", result)

Evaluting Epoch: 1:  10%|█         | 1/10 [00:01<00:14,  1.62s/it]

Names      : ['NLP_CHEST_031']
Predictions: ['']
Labels     : ['Chest PA view : Impression : - Increased both lung markings. - Compatible with right lung nodules. - Fibrotic lesion in left upper lung. - Normal heart size. Atherosclerotic aorta. - Suspect right pleural effusion. - Spondylosis. R / O osteoporosis. - Compatible with multiple bony metastases. - S / P NG and endotracheal tube. - S / P Lt subclavian Port - A - cath.']



Evaluting Epoch: 1:  20%|██        | 2/10 [00:03<00:12,  1.59s/it]

Names      : ['NLP_CHEST_011']
Predictions: ['']
Labels     : ["Chest AP view showed : 1. s / p sternotomy and CABG. Enlarged heart size with tortuous aorta. 2. R't middle and lower lung faint patches. L't lower lung consolidation. L't pleural effusion. 3. No mediastinum widening. 4. s / p endotracheal tube and NG intubation."]



Evaluting Epoch: 1:  30%|███       | 3/10 [00:04<00:10,  1.43s/it]

...

Names      : ['NLP_CHEST_085']
Predictions: ['Chest plain film view showed : Impression : - Increased infiltrations in both lungs. - Blunting left CP angle. - Normal heart size. - Atherosclerotic aorta. - DJD of spine. - S / P NG and endotracheal tube. S / P Rt jugular CVC insertion.']
Labels     : ['Chest X ray : - Right pneumothorax with pleural effusion, status post drainge tube placement. - Extensive subcutaneous emphysema from neck to right side thoracic cage. - Increased right hilar opacity. - Atherosclerosis and tortuous aorta. - Obscured bilateral costophrenic angles. - Spondylosis and mild scoliosis of thoracolumbar spine. - OA of right glenohumeral joint. - Generalized osteopenia.']



Evaluting Epoch: 30:  90%|█████████ | 9/10 [00:12<00:01,  1.27s/it]

Names      : ['NLP_CHEST_027']
Predictions: ['Chest plain film shows : Impression : - Suspicious right upper lung patch. - Increased infiltrations in both lungs. - Blunting bilateral CP angles. - Atherosclerotic aorta. - Scoliosis and DJD of spine. - Bilateral apical pleural thickening. - S / P NG and endotracheal tube.']
Labels     : ['Chest PA View : Impression : > Cardiomegaly with bilateral pulmonary congestion. > Postinflammatory fibrosis in both upper lungs. > Atherosclerosis of aorta. > Old fractures of left 5th and 6th ribs. > R / O osteoporosis. > Spondylosis of thoracolumbar spine. > S / P abdominal operation in RUQ.']



Evaluting Epoch: 30: 100%|██████████| 10/10 [00:13<00:00,  1.34s/it]

Names      : ['NLP_CHEST_071']
Predictions: ['Chest plain film shows : Impression : - Increased infiltrations in both lungs. - Blunting bilateral CP angles. - Tortuous atherosclerotic dilated aorta. - Cardiomegaly. - DJD of spine. - S / P NG and endotracheal tube. S / P Rt jugular CVC insertion.']
Labels     : ['Chest film shows : Impression : - Bilateral lung infiltrations. - Suspect right lower lung patch. Blunting right CP angle. - Tortuous atherosclerotic aorta. - Scoliosis, DJD and osteoporosis of spine. Compression fracture of T12. Old fracture of left ribs. - S / P fixation in L - spine. - S / P tracheostomy and NG tube.']

Rouge-2 score on epoch 29: {'Rouge-L-P': 0.3524809996895963, 'Rouge-L-R': 0.31644925997667717, 'Rouge-L-F': 0.31644925997667717, 'Rouge-2-P': 0.23224644011153853, 'Rouge-2-R': 0.2043349967517373, 'Rouge-2-F': 0.2043349967517373}


In [40]:
import pandas as pd
results = pd.DataFrame(rouge_result)
results

,Rouge-L-P,Rouge-L-R,Rouge-L-F,Rouge-2-P,Rouge-2-R,Rouge-2-F
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.076471,0.084384,0.084384,0.036980,0.038188,0.038188
3,0.116153,0.137362,0.137362,0.060555,0.069650,0.069650
4,0.146011,0.167390,0.167390,0.075101,0.083861,0.083861
5,0.165988,0.183030,0.183030,0.083622,0.089256,0.089256
6,0.190250,0.199816,0.199816,0.100797,0.100932,0.100932
7,0.202408,0.207379,0.207379,0.108528,0.105012,0.105012
8,0.210945,0.217672,0.217672,0.113037,0.112265,0.112265
9,0.220641,0.224434,0.224434,0.121851,0.119165,0.119165


In [ ]:
#model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained("nickmuchi/vit-finetuned-chest-xray-pneumonia", "bert-base-cased").to(device)
#model.load_state_dict(torch.load('outputs/best_checkpoint.pth'))

In [43]:
results = []
model.eval()

pbar = tqdm(valid_dataloader)
pbar.set_description(f"Evaluting Epoch: {epoch + 1}")

with torch.no_grad():
    for names, pixel_values, label_ids in pbar:
        predictions = model.generate(pixel_values,max_length=100)

        _predictions = tokenizer.batch_decode(
                predictions,
                skip_special_tokens=True
        )
        results.append(_predictions)

Evaluting Epoch: 30: 100%|██████████| 10/10 [00:16<00:00,  1.61s/it]


In [44]:
results

[['Chest PA view : Impression : - Increased both lung markings. - Compatible with right lung nodules. - Fibrotic lesion in left upper lung. - Normal heart size. Atherosclerotic aorta. - Suspect right pleural effusion. - Spondylosis. R / O osteoporosis. - Compatible with multiple bony metastases. - S / P N'],
 ["Chest AP view showed : 1. Normal heart size with tortuous aorta. 2. R't middle lung and L't lower lung reticulonodular infiltrations. Mild L't pleural effusion. 3. No mediastinum widening. 4. s / p endotracheal tube and NG intubation. s / p R't jugular CVP."],
 ['Chest PA view shows : Large mass occupying right upper chest. Normal heart size.'],
 ['Chest PA view shows : Impression : > No obvious lung mass nor consolidation patch. > Normal heart size. > No pleural effusion.'],
 ['Chest PA view shows : Impression : - Increased infiltrations in both lungs. - Blunting bilateral CP angles. - Normal heart size. - Atherosclerotic aorta. - DJD of spine. - S / P NG and endotracheal tube.

In [45]:
import csv

# List of strings

# Open or create a CSV file in write mode
with open('submission.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write all strings in a single row
    writer.writerow(results)

print("Data has been written to output.csv")

Data has been written to output.csv
